PEÑA CASTAÑO Javier, HERRERA NATIVI Vladimir

# Projet détection de symptomes radiologiques sur des X-rays 
 

## Structure du Rapport Notebook

Notre rapport sera présenté sous forme de notebook Jupyter ayant au fur et a mesure des explications grace a des markdownds, cette impkémentation nous permettra un projet plus interatifc et guidée. 
1. **Eplications** (cellules Markdown)  
   - Explications des méthodes  
   - Justifications des choix  
   - Analyse interprétative des résultats  

2. **Preuves exécutables** (cellules de code)  




## Contexte et Motivation  
Dans le domaine de la santé, l'intelligence artificielle ouvre des perspectives prometteuses pour l'aide au diagnostic, notamment dans l'analyse d'imagerie médicale. Ce projet s'inscrit dans cette idée, voulant exploiter le dataset NIH Chest X-ray 14 ([disponible sur Kaggle](https://www.kaggle.com/datasets/nih-chest-xrays/data)), contenant **112 120 radiographies thoraciques** annotées avec des informations cliniques et démographiques.  

Notre objectif ? 
- Eliminer les possibles biais par rapport a deux attributs protégés, "Patient Gender" et "Age" 
- Etudier les métriques associés au différents modèles

## Méthodologie Globale  
Le projet s'articule en deux phases complémentaires :  

1. **Pre-processing** :  
   - Rééchantillonnage/Réponderation équitable (Reweighting, DIR, Uniform Sampling)  
 

2. **Post-processing** :  
   - Ajustement des seuils de classification (Reject Option)  
   - Calibration des probabilités (Equalized Odds)  

L'évaluation s'appuie systématiquement sur la librairie AIF360.

## Imports


In [39]:
#IMPORTS
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

# Datasets
import os 
from aif360.datasets import StandardDataset
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.sklearn.metrics import *


#graphiques
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Modèles d'entrainement 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from aif360.algorithms.preprocessing import *

## Chargement des données

In [40]:
data_dir = "HERRERA_NATIVI_VLADIMIR"
metadata_Vlad = pd.read_csv("HERRERA_NATIVI_VLADIMIR/metadata.csv")
metadata_Javi = pd.read_csv("PENA_CASTANO_JAVIER/metadata.csv")
# La prof ma parlé d'un tensorboard

In [41]:
print(f"Vladi : {metadata_Vlad.shape} | Javi : {metadata_Javi.shape}")

Vladi : (5627, 13) | Javi : (5477, 12)


### Choix des données utilisées:

On a choisi les données de vladimir simplement car il y a plus d'entrées, puis on a éliminé colonnes inutiles pour notre études et les lignes impossibles, incohérentes ou avec des valeurs incomplètes.

In [42]:
# Elimination des colonnes non necessaire pour notre étude initial
colonnes_a_eliminer = ['OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]']
data = metadata_Vlad.drop(columns=colonnes_a_eliminer, inplace=False) 
data = data.loc[data['Patient Age'] <= 120] # On élimine les patients qui ont plus de 120 ans 
data.dropna()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train
0,00000006_000.png,No Finding,0,6,81,M,PA,1,True
1,00000025_000.png,Effusion,0,25,71,M,PA,1,True
2,00000029_000.png,No Finding,0,29,59,F,PA,1,False
3,00000072_000.png,Atelectasis,0,72,67,F,PA,1,True
4,00000090_000.png,No Finding,0,90,67,F,PA,1,True
...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,No Finding,0,30752,64,F,AP,1,True
5623,00030772_000.png,No Finding,0,30772,26,F,AP,1,True
5624,00030772_001.png,Consolidation,1,30772,26,F,AP,1,True
5625,00030772_002.png,Consolidation,2,30772,26,F,AP,1,True


# Analyse quantitative des données 

In [43]:
data_graphiques = data.copy() 

On a crée plusieurs graphiques pour voir la repartition des données en fonction de l'age, le sexe et la position de vue.

In [44]:
#Affichage de graphiques pour la visualisation de la repartition de l'age, le follow up, le genre et la position
fig = make_subplots(rows = 2, cols=2, subplot_titles = ["Follow Up", "Age", "Gender", "Position"])
fig.add_trace (go.Histogram(x=data_graphiques['Follow-up #'], name= "follow up"), row = 1, col=1)
fig.add_trace (go.Histogram(x=data_graphiques['Patient Age'], name= "Age"), row = 1, col=2)
fig.add_trace (go.Histogram(x=data_graphiques['Patient Gender'], name= "Gender"), row = 2, col=1)
fig.add_trace (go.Histogram(x=data_graphiques['View Position'], name= "Position"), row = 2, col=2)

fig.update_layout(title = "Histograms")
fig.write_image("Histogrammes1.png")
fig.show()

Ici, on fait de même, mais on normalise les données et on sépare les personnes malades et les personnes saines.

Normaliser et séparer les données va nous permettre de comparer la proportion de personnes malades et saines en fonction de l'age, du genre et de la position de vue. Ceci nous a permit de répérer les attributs favorisés et les non favorisés: les attributs favorisés sont les femmes, PA et les jeunes (0-40 ans). Comme notre label a analyser est "Finding Labels" et notre valeur a favoriser est être sain ( "No Finding"), les attributs favorisés serons les attributs qui on la proportion sains/malades la plus haute.

In [45]:
# Créons d'abord une colonne pour identifier les patients malades et non malades
data_graphiques['Est_Malade'] = ~data_graphiques['Finding Labels'].str.contains('No Finding')
data_graphiques['status'] = data_graphiques['Est_Malade'].map({True: 'Malade', False: 'Non malade'})

# Histogramme pour l'âge
fig = make_subplots(rows=1, cols=3, subplot_titles=["Âge", "Genre", "Position"])

# Groupes d'âge
age_bins = [0, 40, 100]  # Définir 4 tranches d'âge
age_labels = ['0-40', '41-100+']
data_graphiques['age_group'] = pd.cut(data_graphiques['Patient Age'], bins=age_bins, labels=age_labels)

# Compte normalisé par groupe d'âge et par statut (malade/non malade)
age_counts = data_graphiques.groupby(['age_group', 'status']).size().unstack(fill_value=0)
age_counts_normalized = age_counts.div(age_counts.sum(axis=0), axis=1)

# Ajout du trace pour l'âge
for status, color in zip(['Malade', 'Non malade'], ['red', 'green']):
    if status in age_counts_normalized.columns:
        fig.add_trace(
            go.Bar(
                x=age_counts_normalized.index,
                y=age_counts_normalized[status],
                name=status,
                marker_color=color
            ),
            row=1, col=1
        )

# Histogramme pour le genre
gender_counts = data_graphiques.groupby(['Patient Gender', 'status']).size().unstack(fill_value=0)
gender_counts_normalized = gender_counts.div(gender_counts.sum(axis=0), axis=1)

for status, color in zip(['Malade', 'Non malade'], ['red', 'green']):
    if status in gender_counts_normalized.columns:
        fig.add_trace(
            go.Bar(
                x=gender_counts_normalized.index,
                y=gender_counts_normalized[status],
                name=status,
                marker_color=color,
                showlegend=False
            ),
            row=1, col=2
        )

# Histogramme pour la position
position_counts = data_graphiques.groupby(['View Position', 'status']).size().unstack(fill_value=0)
position_counts_normalized = position_counts.div(position_counts.sum(axis=0), axis=1)

for status, color in zip(['Malade', 'Non malade'], ['red', 'green']):
    if status in position_counts_normalized.columns:
        fig.add_trace(
            go.Bar(
                x=position_counts_normalized.index,
                y=position_counts_normalized[status],
                name=status,
                marker_color=color,
                showlegend=False
            ),
            row=1, col=3
        )

# Mise à jour de la mise en page
fig.update_layout(
    title="Distribution normalisée par catégorie (Malades vs Non malades)",
    height=500,
    width=1200,
    bargap=0.1,
    bargroupgap=0.2,
)

fig.update_yaxes(title_text="Proportion", range=[0, 1])
fig.update_xaxes(title_text="Groupe d'âge", row=1, col=1)
fig.update_xaxes(title_text="Genre", row=1, col=2)
fig.update_xaxes(title_text="Position", row=1, col=3)

# Sauvegarde de l'image
fig.write_image("Histogrammes_normalises.png")
fig.show()

/tmp/ipykernel_8024/807476755.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Encodage des dataframes

In [46]:
# Encodage binaire pour Patient Gender et View Position
data['Patient Gender'] = (data['Patient Gender'] == 'F').astype(int) # F est associé a la valeur 1 
data['View Position'] = (data['View Position'] == 'AP').astype(int) # AP est associé a la valeur 1  

# On effectue un encodage binaire pour finding labels du fait qu'on s'interesse uniquement au fait d'être malade ou pas (on n'essaye pas de prédire les maladies)
data['Finding Labels'] = (data['Finding Labels'] == 'No Finding').astype(int) # No Finding est associé a la valeur 1 


In [47]:

# Sauvegarder une fois que le pretraitement sera fait 
data.to_csv(f"{data_dir}/metadata_encod.csv", index=False) # sauvegarde de csv encodé 

# Pre processing

## Première methode naïve de pre-processing : reweighting

### Première aproche a la main pour "Patient Gender"

On commence avec le pre-processing, et on utilise la méthode du cours "Repondération des données".

Pour bien comprendre l'algorithme utilisé, on choisi de faire à la main le premier calcul de poids pour le genre.


In [48]:

# -------------------------------------------  Répondération en fonction du genre  -------------------------------------------
nbtotSexe =  (data["Patient Gender"]).count()
nbFemmes = (data["Patient Gender"] == 1).sum()
nbHommes = (data["Patient Gender"] == 0).sum()
nbMalades = (data["Finding Labels"] == 0).sum()
nbSains = nbtotSexe - nbMalades

# études pour les femmes
nbFemmesMalades = ((data["Patient Gender"] == 1) & (data["Finding Labels"] == 0)).sum()
nbFemmesSain = nbFemmes - nbFemmesMalades
propFMF = nbFemmesMalades / nbFemmes
propFM = nbFemmesMalades / nbtotSexe
propFS = 1 - propFM
propFSF = 1 - propFMF

# étude pour les hommes
nbHommesMalades = ((data["Patient Gender"] == 0) & (data["Finding Labels"] == 0)).sum()
nbHommesSain = nbHommes - nbHommesMalades
propHMH = nbHommesMalades / nbHommes  # proportions des Hommes malades sur popHommes 
propHM = nbHommesMalades / nbtotSexe 
propHS = 1 - propHM
propHSH = 1 - propHMH  # proportions des Hommes sains sur popHommes 


poidsFM = (nbFemmes * nbMalades) / (nbFemmesMalades * nbtotSexe)
poidsFS = (nbFemmes * nbSains) / (nbFemmesSain * nbtotSexe)

poidsHM = (nbHommes * nbMalades) / (nbHommesMalades * nbtotSexe)
poidsHS = (nbHommes * nbSains) / (nbHommesSain * nbtotSexe)

print(f"Poids calculées pour les sexes :\nFemme Malade : {poidsFM}\nFemme Saine : {poidsFS}\nHomme Malade : {poidsHM}\nHomme Sain : {poidsHS}\n")

Poids calculées pour les sexes :
Femme Malade : 1.033302225044209
Femme Saine : 0.9734456018962356
Homme Malade : 0.9754662395970608
Homme Sain : 1.0217508263870834



### Implémentation de l'algorithme automatisé

Ensuite, après avoir compris l'algo, on a mit en place une fonction "re_sampling_naive", qui prend en argument un dataframe D, un attribut S, et le label ("Finding Labels" dans ce cas).

Après avoir appliquer l'algorithme à la position de vue et a l'age, les nouveau poids on était calculés et enregistrés dans u tableau. D'autre part, on a affecté les poids a des variables à la main.

In [49]:
# Favorisés : Jeunes (0-20 ans), Femmes (F), PA

# Défavorisés : Adultes (41-80 ans),Hommes(M),AP

def groupes_bw(S):
    if S == "Patient Gender":
        return [0, 1]
    elif S == "View Position":
        return [1, 0]
    elif S == "Age Group":
        return [1, 0]


def re_sampling_naive(D, S, Label):
    W = []
    groupes = groupes_bw(S)
    for s in groupes:
        for c in [0, 1]:  # classes - et +
            num = ((D[S] == s).sum())*((D[Label]==c).sum())
            denom = (((D[S] == s) & (D[Label] == c)).sum()) * D.shape[0]
            poid =   num / denom
            W.append(poid)
    return W


In [50]:
poids_VP = re_sampling_naive(data,"View Position", "Finding Labels" )

poidsAPM = poids_VP[0]
poidsAPS = poids_VP[1]

poidsPAM = poids_VP[2]
poidsPAS = poids_VP[3]
poids_VP

[np.float64(0.8660089311858132),
 np.float64(1.150692505153632),
 np.float64(1.1149736044363807),
 np.float64(0.9197266785927081)]

En vue que les ages ne sont pas binaires, on a crée un attribut de data 'Age Group', qui sépare les jeunes (0 - 39 ans) et les vieux (40 - 120), puis on applique l'algo a cet attribut.

In [51]:
seuil_age = 40
data['Age Group'] = np.where(data['Patient Age'] >= seuil_age, 1, 0) # 1 répresent les "vieux"

poids_AGE = re_sampling_naive(data,"Age Group", "Finding Labels" )

poidsVieuxM = poids_AGE[0]
poidsVieuxS = poids_AGE[1]

poidsJeunesM = poids_AGE[2]
poidsJeunesS = poids_AGE[3]
poids_AGE

[np.float64(0.9855372894715779),
 np.float64(1.0125771911382422),
 np.float64(1.0322783050765911),
 np.float64(0.9742161286775578)]

Après affecter toutes les poids calculés, on introduit 3 colonnes avec les poids calculés a notre dataframe non encodé.

In [52]:
# Création des nouvelles colonnes avec des valeurs par défaut
# On supprime les colonnes inutiles et on filtre les patients de plus de 120 ans
data_c = metadata_Vlad.drop(columns=colonnes_a_eliminer, inplace=False)  
data_c = data_c.loc[data_c['Patient Age'] <= 120]  # Élimine les patients âgés de plus de 120 ans

# Création des colonnes de poids avec des valeurs par défaut à 1.0
data_c['Age Group'] = np.where(data_c['Patient Age'] >= seuil_age, 1, 0)  # Binarisation de l'âge (1 = vieux, 0 = jeune)
data_c['poids_reweigth_gender'] = 1.0  # Poids par défaut pour le genre
data_c['poids_reweigth_PA'] = 1.0      # Poids par défaut pour la position (AP/PA)
data_c['poids_reweigth_age'] = 1.0      # Poids par défaut pour l'âge

# Attribution des poids en fonction du genre et de l'état de santé
# Pour les femmes (Genre = "F")
data_c.loc[(data_c['Patient Gender'] == "F") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_gender'] = poidsFM  # Femmes malades
data_c.loc[(data_c['Patient Gender'] == "F") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_gender'] = poidsFS    # Femmes saines

# Pour les hommes (Genre = "M")
data_c.loc[(data_c['Patient Gender'] == "M") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_gender'] = poidsHM  # Hommes malades
data_c.loc[(data_c['Patient Gender'] == "M") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_gender'] = poidsHS    # Hommes sains

# Attribution des poids en fonction de la position (AP/PA) et de l'état de santé
# Pour la position AP (View Position = "AP")
data_c.loc[(data_c['View Position'] == "AP") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_PA'] = poidsAPM  # AP malade
data_c.loc[(data_c['View Position'] == "AP") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_PA'] = poidsAPS    # AP sain

# Pour la position PA (View Position = "PA")
data_c.loc[(data_c['View Position'] == "PA") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_PA'] = poidsPAM  # PA malade
data_c.loc[(data_c['View Position'] == "PA") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_PA'] = poidsPAS    # PA sain

# Attribution des poids en fonction de l'âge et de l'état de santé
# Pour les patients âgés (Age Group = 1)
data_c.loc[(data_c['Age Group'] == 1) & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_age'] = poidsVieuxM  # Âgé malade
data_c.loc[(data_c['Age Group'] == 1) & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_age'] = poidsVieuxS    # Âgé sain

# Pour les patients jeunes (Age Group = 0)
data_c.loc[(data_c['Age Group'] == 0) & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_age'] = poidsJeunesM  # Jeune malade
data_c.loc[(data_c['Age Group'] == 0) & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_age'] = poidsJeunesS    # Jeune sain

# Sauvegarde du nouveau jeu de données avec les poids
data_c.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_c_with_weights.csv", index=False)



## Uniform-Sampling reweighting

Maintenant on s'intéresse a un deuxième algorithme du cours: Uniform sampling.

In [53]:
# Favorisés : Jeunes (0-20 ans), Femmes (F), PA

# Défavorisés : Adultes (41-80 ans), Hommes (M), AP

def uniform_sampling(D, S, Label):
    W = []
    groupes = groupes_bw(S)
    for s in groupes:
        for c in [0, 1]:
            num = ((D[S] == s).sum())*((D[Label]==c).sum())
            denom = (((D[S] == s) & (D[Label] == c)).sum()) * D.shape[0]
            poid = num / denom
            W.append(poid)

    nb_DN = ((D[S] == groupes[0]) & (D[Label] == 0)).sum()
    nb_DP = ((D[S] == groupes[0]) & (D[Label] == 1)).sum()
    nb_FN = ((D[S] == groupes[1]) & (D[Label] == 0)).sum()
    nb_FP = ((D[S] == groupes[1]) & (D[Label] == 1)).sum()

    nb_samples_DN = int(W[0]*nb_DN)
    nb_samples_DP = int(W[1]*nb_DP)
    nb_samples_FN = int(W[2]*nb_FN)
    nb_samples_FP = int(W[3]*nb_FP)

    samples_DN = D[(D[S] == groupes[0]) & (D[Label] == 0)].sample(n=nb_samples_DN, replace=True)  
    samples_DP = D[(D[S] == groupes[0]) & (D[Label] == 1)].sample(n=nb_samples_DP, replace=True)
    samples_FN = D[(D[S] == groupes[1]) & (D[Label] == 0)].sample(n=nb_samples_FN, replace=True)
    samples_FP = D[(D[S] == groupes[1]) & (D[Label] == 1)].sample(n=nb_samples_FP, replace=True)
    
    df_resampled = pd.concat([samples_DN, samples_DP, samples_FN, samples_FP])

    return df_resampled

On a constaté que l'attribut "View Position" ne peut pas être un biais car ça n'as pas de sense de discriminer quelqu'un par la "View Position".

Ici, on affecte "df_sexe" et "df_age" avec Uniform Sampling, et on sauvegarde le dataframe pour l'entreiner plus tard dans le fichier "train_classifieur.ipynb".

In [54]:
# Favorisés : Jeunes (0-20 ans), Femmes (F), PA
# Défavorisés : Adultes (41-80 ans), Hommes (M), AP

df_sexe = uniform_sampling(data, "Patient Gender", "Finding Labels")
df_age = uniform_sampling(data, "Patient Gender", "Finding Labels")


df_sexe.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_uniform_genre.csv", index=False)
df_age.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_uniform_age.csv", index=False)

# AIF360

On veut maintenant appliquer les méthodes de fairness en utilisant directement la bibliothèque AIF360.

Dans un premier temps, on fait une table de correspondance pour pouvoir recupérer les "Image Index", pour pouvoir bien entraîner nas modèles.

In [55]:
# Transformation du type de la colonne Image Index (string) en int et création de la table de correspondance 

mapping_dict = {val: idx for idx, val in enumerate(data['Image Index'].unique())}
data['Image Index'] = data['Image Index'].map(mapping_dict)
mapping_table = pd.DataFrame(list(mapping_dict.items()), columns=['Ancien Image Index', 'Nouveau Index'])


Ensuite, on diviser notre dataframe en 2: Validation et Entraînement

In [56]:
data_train = data[data["train"] == True].copy()
data_val = data[data["train"] == False].copy()

### Creer une instance de StandardDataset de AIF360

On encode nos données au cas où il ne sont pas encore encodés pour pouvoir ensuite transformer notre dataframe en DataSet.

In [57]:
# Création d'un dictionnaire pour gérer les colonnes catégorielles encodées en one-hot
# Ce dictionnaire va associer chaque nom de colonne catégorielle originale
# à la liste des colonnes one-hot correspondantes dans le DataFrame

categorical_columns_dic = {}
for col in data.columns:
    col_split = col.split("=")
    if len(col_split) > 1:
        cat_col = col_split[0]
        if not (cat_col in categorical_columns_dic.keys()):
            categorical_columns_dic[cat_col] = []
        categorical_columns_dic[cat_col].append(col)
categorical_features = categorical_columns_dic.keys()

### Transformation des dataFrames en Dataset

Comme dans les TDs, on trans les dataFrames de train, valid et général en Dataset, prennant en considération les attributs sur lesquels on va travailler, le label utilisé, la classe favorable et les classes privilégiés.

In [58]:
MyDataset_train = StandardDataset(
    df=data_train,
    label_name="Finding Labels",
    favorable_classes=[1],
    protected_attribute_names=["Patient Gender", "Age Group"],
    privileged_classes=[[1],[0]], # a analyser 
    categorical_features=categorical_features,
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

MyDataset_val = StandardDataset(
    df=data_val,
    label_name="Finding Labels",
    favorable_classes=[1],
    protected_attribute_names=["Patient Gender","Age Group"],
    privileged_classes=[[1],[0]], # a analyser 
    categorical_features=categorical_features,
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

MyDataset = StandardDataset(
    df=data,
    label_name="Finding Labels",
    favorable_classes=[1],
    protected_attribute_names=["Patient Gender","Age Group"],
    privileged_classes=[[1],[0]], # a analyser 
    categorical_features=categorical_features,
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

## Pre-processing : Implémentations Reweighing AIF360

Dans cette partie, on a constaté que le Reweighing AIF360 effectué le même calcul qu'on a fait au début du pre-processing. 
Cependant, pour la méthode avec AIF360, on a séparé l'entrainement et la validation, et c'est pour cela qu'on a des poids diférents qu'au début.

In [59]:
sens_ind = 0
sens_attr = MyDataset_train.protected_attribute_names[sens_ind]
unprivileged_groups = [
    {sens_attr: v}
    for v in MyDataset_train.unprivileged_protected_attributes[sens_ind]
]
privileged_groups = [
    {sens_attr: v}
    for v in MyDataset_train.privileged_protected_attributes[sens_ind]
]
sens_attr, unprivileged_groups, privileged_groups

('Patient Gender',
 [{'Patient Gender': np.float64(0.0)}],
 [{'Patient Gender': np.float64(1.0)}])

#### Reweighing sur le genre

Pour les deux cas de Reweighing, on affecte les trois variables dataset_transf_train, dataset_transf_val et dataset_transf, après avoir entrainé notre model. 
"dataset_transf" va nous permettre d'alloué les nouveaux poids calculés pour l'entrainement sans probèmes de taille dans notre fichier csv avec le reste des poids.

In [60]:
RW_gender = Reweighing(
    unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups
)

RW_gender.fit(MyDataset_train)

dataset_transf = RW_gender.transform(MyDataset)

dataset_transf_train = RW_gender.transform(MyDataset_train)
dataset_transf_val = RW_gender.transform(MyDataset_val)

#### Reweighing sur l'âge

In [61]:
sens_ind_age = 1
sens_attr_age = MyDataset_train.protected_attribute_names[sens_ind_age]
unprivileged_groups_age = [
    {sens_attr_age: v}
    for v in MyDataset_train.unprivileged_protected_attributes[sens_ind_age]
]
privileged_groups_age = [
    {sens_attr_age: v}
    for v in MyDataset_train.privileged_protected_attributes[sens_ind_age]
]
sens_attr_age, unprivileged_groups_age, privileged_groups_age

('Age Group',
 [{'Age Group': np.float64(1.0)}],
 [{'Age Group': np.float64(0.0)}])

In [62]:
RW_age = Reweighing(
    unprivileged_groups=unprivileged_groups_age, privileged_groups=privileged_groups_age
)

RW_age.fit(MyDataset_train)

dataset_transf_age = RW_age.transform(MyDataset)

dataset_transf_age_train = RW_age.transform(MyDataset_train)
dataset_trans_age_val = RW_age.transform(MyDataset_val)


On introduit deux colonnes avec les poids calculés a notre dataframe non encodé, de manière "automatique".

In [63]:
# Ajout des weights calculés avec aif360
data_c["weights_gender_aif360"] = dataset_transf.instance_weights
data_c["weights_age_aif360"] = dataset_transf_age.instance_weights
# Sauvegarder data_c mit à jour avec les nouvelles colonnes
data_c.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_c_with_weights.csv", index=False)
data_c


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train,Age Group,poids_reweigth_gender,poids_reweigth_PA,poids_reweigth_age,weights_gender_aif360,weights_age_aif360
0,00000006_000.png,No Finding,0,6,81,M,PA,1,True,1,1.021751,0.919727,1.012577,0.993727,0.997075
1,00000025_000.png,Effusion,0,25,71,M,PA,1,True,1,0.975466,1.114974,0.985537,1.008043,1.003722
2,00000029_000.png,No Finding,0,29,59,F,PA,1,False,1,0.973446,0.919727,1.012577,1.007274,0.997075
3,00000072_000.png,Atelectasis,0,72,67,F,PA,1,True,1,1.033302,1.114974,0.985537,0.990955,1.003722
4,00000090_000.png,No Finding,0,90,67,F,PA,1,True,1,0.973446,0.919727,1.012577,1.007274,0.997075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,No Finding,0,30752,64,F,AP,1,True,1,0.973446,1.150693,1.012577,1.007274,0.997075
5623,00030772_000.png,No Finding,0,30772,26,F,AP,1,True,0,0.973446,1.150693,0.974216,1.007274,1.005441
5624,00030772_001.png,Consolidation,1,30772,26,F,AP,1,True,0,1.033302,0.866009,1.032278,0.990955,0.993207
5625,00030772_002.png,Consolidation,2,30772,26,F,AP,1,True,0,1.033302,0.866009,1.032278,0.990955,0.993207


## Disparate Impact Remover 

Une seule méthode de préprocessing de AIF360 nous semblait pas suffisant, donc on a implémente une deuxième méthode: Disparate Impact Remover. Appliqué de même sur le genre et l'age comme attributs, et suivant la méthode vue en cours, on a obtenue deux nouveaux datasets, avec lequels on va pouvoir comparer les metriques.

In [64]:
# Disparate Impact Remover : PATIENT GENDER 

DIR_gender = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Patient Gender")

DIR_gender.fit(MyDataset_train)

dataset_transf_dir_gender = DIR_gender.fit_transform(MyDataset)
dataset_transf_dir_gender_train = DIR_gender.fit_transform(MyDataset_train)
dataset_transf_dir_gender_val = DIR_gender.fit_transform(MyDataset_val)


In [65]:
# Disparate Impact Remover : AGE GROUP

DIR_age = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Age Group")
DIR_age.fit(MyDataset_train)

dataset_transf_dir_age = DIR_age.fit_transform(MyDataset)
dataset_transf_dir_age_train = DIR_age.fit_transform(MyDataset_train)
dataset_transf_dir_age_val = DIR_age.fit_transform(MyDataset_val)



Tout au long de notre projet, on a été plus à laise avec les dataframe qu'avec les datasets, du coup on a choisi de transformer les datasets en dataframe à nouveaux.

In [66]:
data_transformed_gender = dataset_transf_dir_gender.convert_to_dataframe()[0]
data_transformed_age = dataset_transf_dir_age.convert_to_dataframe()[0]


# Ajouter les données transformées au DataFrame existant
# Vous pouvez ajouter les colonnes nécessaires de `data_transformed_gender` au DataFrame `data_dir_gender`
data_dir_gender = data_transformed_gender.copy()
data_dir_age = data_transformed_age.copy()

data_dir_age

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train,Age Group,Finding Labels
0,0.0,0.0,6.0,32.0,0.0,0.0,1.0,1.0,1.0,1.0
1,0.0,0.0,6.0,25.0,0.0,0.0,1.0,1.0,1.0,0.0
2,2.0,0.0,29.0,16.0,1.0,0.0,1.0,0.0,1.0,1.0
3,2.0,0.0,29.0,22.0,1.0,0.0,1.0,1.0,1.0,0.0
4,4.0,0.0,90.0,22.0,1.0,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5622,5619.0,0.0,30711.0,19.0,1.0,1.0,1.0,1.0,1.0,1.0
5623,5610.0,0.0,30687.0,26.0,1.0,1.0,1.0,1.0,0.0,1.0
5624,5613.0,1.0,30687.0,26.0,1.0,1.0,1.0,1.0,0.0,0.0
5625,5617.0,2.0,30687.0,26.0,1.0,1.0,1.0,1.0,0.0,0.0


Suite à la transformation, on devait modifier les valeurs des "Image Index" aux valeurs originals (pour ne pas avoir d'erreurs dans l'entrainement), en utilisant les dictionnaire crée auparavant.

In [67]:
# Créer un dictionnaire inverse où les clés sont les nouveaux index et les valeurs sont les anciens noms
inverse_mapping = {v: k for k, v in mapping_dict.items()}

# Appliquer ce mapping inverse à data_dir_gender et à data_dir_age pour retrouver les noms originaux
data_dir_gender['Image Index'] = data_dir_gender['Image Index'].map(inverse_mapping)
data_dir_age['Image Index'] = data_dir_age['Image Index'].map(inverse_mapping)


In [68]:
data_dir_gender.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_DIR_genre.csv", index=False)
data_dir_age.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_DIR_age.csv", index=False)

## Cacul de metriques suite au pre-processing

Implémntation de la fonction de cours, pour avoir le maximun de métriques possibles.

In [69]:
def get_group_metrics(
    y_true,
    y_pred=None,
    prot_attr=None,
    priv_group=1,
    pos_label=1,
    sample_weight=None
):
    group_metrics = {}
    
    # Base rate
    group_metrics["base_rate"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    
    # Check for valid protected attribute
    if prot_attr is None:
        raise ValueError("prot_attr must be specified for fairness metrics")
    
    # Statistical parity difference
    try:
        group_metrics["statistical_parity_difference"] = statistical_parity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, 
            priv_group=priv_group, pos_label=pos_label, 
            sample_weight=sample_weight
        )
    except ZeroDivisionError:
        group_metrics["statistical_parity_difference"] = 0.0
    
    # Disparate impact ratio
    try:
        group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, 
            priv_group=priv_group, pos_label=pos_label, 
            sample_weight=sample_weight
        )
    except ZeroDivisionError:
        group_metrics["disparate_impact_ratio"] = 0.0
    
    if y_pred is not None:
        # Equal opportunity difference
        try:
            group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, 
                priv_group=priv_group, pos_label=pos_label, 
                sample_weight=sample_weight
            )
        except ZeroDivisionError:
            group_metrics["equal_opportunity_difference"] = 0.0
        
        # Average odds difference
        try:
            group_metrics["average_odds_difference"] = average_odds_difference(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, 
                priv_group=priv_group, pos_label=pos_label, 
                sample_weight=sample_weight
            )
        except ZeroDivisionError:
            group_metrics["average_odds_difference"] = 0.0
    
    return group_metrics

## Analyse des métriques de fairness

Dans cette partie, nous évaluons l'impact de différentes méthodes de pré-processing sur la fairness de notre modèle à travers plusieurs métriques :

1. **Métriques originales** : Baseline sans aucun traitement de fairness
2. **Reweighing** : Rééquilibrage par poids pour le genre et l'âge
3. **Disparate Impact Remover** : Correction des impacts disproportionnés
4. **Uniform Sampling** : Rééchantillonnage uniforme des groupes

Les métriques calculées pour chaque méthode incluent (à cause du manque de predictions):
- Base rate (taux de base)
- Statistical parity difference (parité statistique) 
- Disparate impact ratio (impact disproportionné)

In [70]:
# Pour les données originales pour le sexe
metrics_orig = get_group_metrics(
    y_true=data["Finding Labels"], 
    prot_attr=data_dir_gender["Patient Gender"],
    priv_group=1,
    pos_label=1,  
    sample_weight = data_c.WEIGHTS
)

# Pour les données originales pour l'age
metrics_orig_age = get_group_metrics(
    y_true=data["Finding Labels"], 
    prot_attr=data_dir_gender["Age Group"],
    priv_group=1,
    pos_label=1,  
    sample_weight = data_c.WEIGHTS
)

# Pour les données transformées par Reweighing pour le sexe
metrics_rw_gender = get_group_metrics(
    y_true=data["Finding Labels"],  
    prot_attr=data["Patient Gender"],  # L'attribut protégé (ici "Patient Gender")
    priv_group=1,
    pos_label=1,  
    sample_weight = data_c.weights_gender_aif360
)

# Pour les données transformées par Reweighing pour l'âge
metrics_rw_age = get_group_metrics(
    y_true=data["Finding Labels"],  
    prot_attr=data["Age Group"], 
    priv_group=0,
    pos_label=1,
    sample_weight=data_c.weights_age_aif360
)

# Pour les données transformées par DisparateImpactRemover pour le sexe
metrics_dir_gender = get_group_metrics(
    y_true=data_dir_gender["Finding Labels"],  
    prot_attr=data_dir_gender["Patient Gender"], 
    priv_group=1,
    pos_label=1,
    sample_weight=data_dir_gender.WEIGHTS
)

# Pour les données transformées par DisparateImpactRemover pour l'âge
metrics_dir_age = get_group_metrics(
    y_true=data_dir_gender["Finding Labels"],  
    prot_attr=data_dir_age["Age Group"], 
    priv_group=0,
    pos_label=1,
    sample_weight=data_dir_age.WEIGHTS
)

# Pour les données transformées par Uniform Sampling pour le sexe
metrics_uniform_gender = get_group_metrics(
    y_true=df_sexe["Finding Labels"],  
    prot_attr=df_sexe["Patient Gender"], 
    priv_group=1,
    pos_label=1,
    sample_weight=df_sexe.WEIGHTS
)

# Pour les données transformées par Uniform Sampling pour l'âge
metrics_uniform_age = get_group_metrics(
    y_true=df_age["Finding Labels"],  
    prot_attr=df_age["Age Group"], 
    priv_group=0,
    pos_label=1,
    sample_weight=df_age.WEIGHTS
)



# Afficher ou analyser les résultats
print("Métriques originales (Gender):", metrics_orig , "\n")
print("Métriques originales (Age):", metrics_orig_age , "\n")
print("Métriques après Reweighing (Gender):", metrics_rw_gender, "\n")
print("Métriques après Reweighing (Age):", metrics_rw_age, "\n")
print("Métriques après Disparate Impact Remover (Gender):", metrics_dir_gender, "\n")
print("Métriques après Disparate Impact Remover (Age):", metrics_dir_age, "\n")
print("Métriques après Uniform Sampling (Gender):", metrics_uniform_gender, "\n")
print("Métriques après Uniform Sampling  (Age):", metrics_uniform_age, "\n")


Métriques originales (Gender): {'base_rate': np.float64(0.5415926057589762), 'statistical_parity_difference': np.float64(-0.02630329442442525), 'disparate_impact_ratio': 0.9527230874266526} 

Métriques originales (Age): {'base_rate': np.float64(0.5415926057589762), 'statistical_parity_difference': np.float64(0.02106104410981824), 'disparate_impact_ratio': 1.0393763368635225} 

Métriques après Reweighing (Gender): {'base_rate': np.float64(0.5413524681936874), 'statistical_parity_difference': np.float64(-0.03389554132548789), 'disparate_impact_ratio': 0.9395148406767924} 

Métriques après Reweighing (Age): {'base_rate': np.float64(0.541431091861975), 'statistical_parity_difference': np.float64(-0.02573446024774384), 'disparate_impact_ratio': 0.9539589999836494} 

Métriques après Disparate Impact Remover (Gender): {'base_rate': np.float64(0.5415926057589762), 'statistical_parity_difference': np.float64(-0.02630329442442525), 'disparate_impact_ratio': 0.9527230874266526} 

Métriques après 

## Analyse détaillée des métriques de fairness

### Résultats par attribut protégé :

**1. Métriques originales :**
- **Genre** : 
  - Parité statistique : -0.026 (léger désavantage pour le groupe non privilégié)
  - Disparate Impact Ratio (DIR) : 0.953 (proche de 1, indicateur d'un biais modéré)
  
- **Âge** :
  - Parité statistique : +0.021 (léger avantage pour le groupe privilégié)
  - DIR : 1.039 (légèrement supérieur à 1, biais inversé)

**2. Après Reweighing :**
- **Genre** :
  - Aggravation du biais (DIR passe de 0.953 à 0.940)
  - Parité statistique plus négative (-0.034)
  
- **Âge** :
  - Inversion du biais (DIR passe à 0.954)
  - Parité statistique devient négative (-0.026)

**3. Après Disparate Impact Remover :**
- **Genre** :
  - Résultats identiques aux métriques originales
  - La méthode n'a pas modifié le biais existant
  
- **Âge** :
  - Amélioration du DIR (1.039 → 0.962)
  - Parité statistique inversée (+0.021 → -0.021)

**4. Après Uniform Sampling :**
- **Genre** :
  - Meilleurs résultats (DIR = 1.000, parité quasi-nulle)
  - Méthode très efficace pour cet attribut
  
- **Âge** :
  - Aggravation du biais (DIR = 0.949)
  - Parité statistique plus négative (-0.029)

### Tendances observées :
1. **Efficacité variable** selon l'attribut :
   - Uniform Sampling excellent pour le genre mais contre-productif pour l'âge
   - Disparate Impact Remover montre l'effet inverse

2. **Impact des méthodes** :
   - Reweighing tend à uniformiser les biais dans la même direction
   - Aucune méthode ne donne des résultats optimaux pour les deux attributs simultanément

3. **Interprétation des DIR** :
   - Valeurs idéales proches de 1
   - Écart plus important pour l'âge que pour le genre


## Conclusion

Cette analyse montre que l'équité algorithmique nécessite des approches spécifiques à chaque attribut protégé, sans solution universelle. Une combinaison de méthodes ciblées semble optimale pour mitiger simultanément les différents biais identifiés.

# Post-processing


## Application des méthodes de post-traitement



## Interprétation des résultats

### Compréhension des métriques clés
- **Statistical Parity Difference (SPD)** : Mesure la disproportion dans les prédictions positives entre groupes
- **Equal Opportunity Difference (EOD)** : Évalue les différences de taux de vrais positifs
- **Average Odds Difference (AOD)** : Combine les disparités de FPR et FNR


## Optimisation par Equalized Odds

### Méthodologie calibrée
Application de la méthode Calibrated Equalized Odds avec différentes contraintes :
- **FNR** : Minimisation des faux négatifs
- **FPR** : Réduction des faux positifs
- **Weighted** : Combinaison pondérée


### Régression logistique et  Reject-option Classification
Pour povoir appliqué la méthode Reject-option Classification (ROC), il nous fallait une régression logistique pour obtenir des probabilités de prédiction. Cette approche consiste à :
- Définir les thresholds de classification optimaux par attribut sensible
- Minimiser les disparités entre groupes privilégiés et non-privilégiés

**Méthodologie appliquée** :
- Entraînement séparé sur chaque attribut protégé (Patiend Gender et Age Group)
- Calcul des divers métriques montré en cours et au TDs 





## Reject-option classification

In [71]:
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.metrics import ClassificationMetric
import pandas as pd

def single_attribute_fairness_analysis(dataset_train, dataset_val, protected_attributes):
    """
    Perform ROC fairness adjustment for each protected attribute separately
    
    Args:
        dataset_train: AIF360 training dataset
        dataset_val: AIF360 validation dataset
        protected_attributes: List of protected attribute names to analyze
        
    Returns:
        Dictionary of fairness metrics per attribute
    """
    # 1. Train base model
    model = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)
    model.fit(dataset_train.features, dataset_train.labels.ravel())
    
    # 2. Get predicted scores
    val_scores = model.predict_proba(dataset_val.features)[:, 1]
    val_pred = dataset_val.copy(deepcopy=True)
    val_pred.scores = val_scores.reshape(-1, 1)
    
    results = {}
    
    for attr in protected_attributes:
        print(f"\n{'='*40}\nAnalyzing attribute: {attr}\n{'='*40}")
        
        # 3. Get attribute-specific groups
        attr_idx = dataset_val.protected_attribute_names.index(attr)
        privileged_group = [{attr: dataset_val.privileged_protected_attributes[attr_idx][0]}]
        unprivileged_group = [{attr: v} for v in dataset_val.unprivileged_protected_attributes[attr_idx]]
        
        # 4. Configure ROC
        roc = RejectOptionClassification(
            unprivileged_groups=unprivileged_group,
            privileged_groups=privileged_group,
            metric_name="Statistical parity difference",
            metric_ub=0.05,
            metric_lb=-0.05,
            low_class_thresh=0.01,
            high_class_thresh=0.99,
            num_class_thresh=50,
            num_ROC_margin=30,
        )
        
        # 5. Fit and predict
        roc.fit(dataset_val, val_pred)
        fair_pred = roc.predict(val_pred)
        
        # 6. Calculate metrics
        metric = ClassificationMetric(
            dataset_val,
            fair_pred,
            unprivileged_groups=unprivileged_group,
            privileged_groups=privileged_group
        )
        
        # Store results
        results[attr] = {
            'threshold': roc.classification_threshold,
            'margin': roc.ROC_margin,
            'spd': metric.statistical_parity_difference(),
            'eod': metric.equal_opportunity_difference(),
            'aod': metric.average_odds_difference()
        }
        
        # Print results
        print(f"\n{attr} fairness metrics:")
        print(f"  Optimal threshold: {results[attr]['threshold']:.4f}")
        print(f"  Statistical parity difference: {results[attr]['spd']:.4f}")
        print(f"  Equal opportunity difference: {results[attr]['eod']:.4f}")
        print(f"  Average odds difference: {results[attr]['aod']:.4f}")
    
    return results

# =================================================================
# Usage with your dataset
# =================================================================

# List of protected attributes to analyze separately
protected_attributes = ["Patient Gender", "Age Group"]

# Run analysis
metrics_results = single_attribute_fairness_analysis(
    MyDataset_train,
    MyDataset_val,
    protected_attributes
)


final_predictions = MyDataset_val.copy(deepcopy=True)
for attr in protected_attributes:
    # Get attribute-specific predictions
    attr_idx = MyDataset_val.protected_attribute_names.index(attr)
    privileged_group = [{attr: MyDataset_val.privileged_protected_attributes[attr_idx][0]}]
    unprivileged_group = [{attr: v} for v in MyDataset_val.unprivileged_protected_attributes[attr_idx]]
    
    roc = RejectOptionClassification(
        unprivileged_groups=unprivileged_group,
        privileged_groups=privileged_group,
        metric_name="Statistical parity difference",
        metric_ub=0.05,
        metric_lb=-0.05
    ).fit(MyDataset_val, final_predictions)
    
    final_predictions = roc.predict(final_predictions)

print("\nFinal combined metrics:")
combined_metric = ClassificationMetric(
    MyDataset_val,
    final_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
print(f"Statistical parity difference: {combined_metric.statistical_parity_difference():.4f}")
print(f"Equal opportunity difference: {combined_metric.equal_opportunity_difference():.4f}")


Analyzing attribute: Patient Gender

Patient Gender fairness metrics:
  Optimal threshold: 0.4900
  Statistical parity difference: 0.0288
  Equal opportunity difference: 0.0466
  Average odds difference: 0.0454

Analyzing attribute: Age Group

Age Group fairness metrics:
  Optimal threshold: 0.4900
  Statistical parity difference: 0.0135
  Equal opportunity difference: 0.0440
  Average odds difference: 0.0242


/home/vladhnat/Fairness/projetFairness/env/lib/python3.10/site-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning:

Unable to satisy fairness constraints




Final combined metrics:
Statistical parity difference: -0.1098
Equal opportunity difference: 0.0000


### Explication de la méthode Reject Option Classification (ROC)

La méthode **Reject Option Classification** vise à améliorer l’équité des prédictions en ajustant les décisions du modèle dans une **zone critique** autour du seuil de classification. Cette zone est particulièrement importante pour les individus appartenant à un **groupe défavorisé**.

Concrètement, pour les cas où la probabilité prédite se trouve dans cette zone, la classe prédite peut être inversée afin de **réduire les biais** (notamment la parité statistique) et favoriser l'équité.

#### Mise en œuvre dans le code

1. **Entraînement initial** :
   - Un modèle de régression logistique est entraîné sur l’ensemble d’apprentissage.

2. **Prédictions de validation** :
   - Les probabilités prédites par le modèle sont utilisées sur l’ensemble de validation.

3. **Définition des groupes protégés** :
   - Pour chaque attribut protégé (ici `Patient Gender` et `Age Group`), les groupes **privilégié** et **non privilégié** sont définis.

4. **Application de la méthode ROC** :
   - Une fenêtre d’acceptation est fixée (par exemple entre -0.05 et 0.05 pour la **Statistical Parity Difference**).
   - Dans cette fenêtre, les prédictions peuvent être ajustées pour améliorer l’équité.

5. **Évaluation** :
   - Les prédictions ajustées sont évaluées à l’aide de plusieurs métriques de biais :
     - **Statistical parity difference**
     - **Equal opportunity difference**
     - **Average odds difference**

#### Résultats obtenus

**Pour l’attribut `Patient Gender`** :
- Seuil optimal appliqué : `0.4900`
- Statistical parity difference : `0.0288`
- Equal opportunity difference : `0.0466`
- Average odds difference : `0.0454`

**Pour l’attribut `Age Group`** :
- Seuil optimal appliqué : `0.4900`
- Statistical parity difference : `0.0135`
- Equal opportunity difference : `0.0440`
- Average odds difference : `0.0242`

Ces résultats indiquent que, **lorsqu'elle est appliquée individuellement**, la méthode ROC **atténue efficacement les disparités** entre groupes protégés. Les valeurs des métriques se rapprochent de zéro, signe d'une meilleure équité.

#### Conclusion

La méthode **Reject Option Classification** permet de **réduire les biais** en ajustant les décisions dans une zone proche du seuil de classification, notamment pour les groupes protégés. Lorsqu'elle est utilisée **séparément sur chaque attribut**, elle donne des résultats prometteurs.

Cependant, appliquer simultanément plusieurs ajustements (pour différents attributs protégés) peut entraîner des **compromis difficiles à satisfaire**. Malgré cela, la ROC reste une approche **efficace** pour améliorer la fainess.


## Calibrated Equalized-Odds

In [72]:
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing

def calibrated_eq_odds_analysis(dataset_train, dataset_val, protected_attributes, cost_constraint="fnr"):
    """
    Perform Calibrated Equalized Odds fairness adjustment for each protected attribute
    
    Args:
        dataset_train: AIF360 training dataset
        dataset_val: AIF360 validation dataset
        protected_attributes: List of protected attribute names
        cost_constraint: Optimization target ("fnr", "fpr", or "weighted")
        
    Returns:
        Dictionary of fairness metrics per attribute
    """
    # 1. Train base model
    model = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)
    model.fit(dataset_train.features, dataset_train.labels.ravel())
    
    # 2. Get predicted scores
    val_scores = model.predict_proba(dataset_val.features)[:, 1]
    val_pred = dataset_val.copy(deepcopy=True)
    val_pred.scores = val_scores.reshape(-1, 1)
    
    results = {}
    
    for attr in protected_attributes:
        print(f"\n{'='*40}\nAnalyzing attribute: {attr}\n{'='*40}")
        
        # 3. Get attribute-specific groups
        attr_idx = dataset_val.protected_attribute_names.index(attr)
        privileged_group = [{attr: dataset_val.privileged_protected_attributes[attr_idx][0]}]
        unprivileged_group = [{attr: v} for v in dataset_val.unprivileged_protected_attributes[attr_idx]]
        
        # 4. Configure Calibrated Equalized Odds
        ceo = CalibratedEqOddsPostprocessing(
            privileged_groups=privileged_group,
            unprivileged_groups=unprivileged_group,
            cost_constraint=cost_constraint,
            seed=42
        )
        
        # 5. Fit and predict
        ceo.fit(dataset_val, val_pred)
        fair_pred = ceo.predict(val_pred)
        
        # 6. Calculate metrics
        metric = ClassificationMetric(
            dataset_val,
            fair_pred,
            unprivileged_groups=unprivileged_group,
            privileged_groups=privileged_group
        )
        
        # Store results
        results[attr] = {
            'spd': metric.statistical_parity_difference(),
            'eod': metric.equal_opportunity_difference(),
            'aod': metric.average_odds_difference(),
            'fnr_diff': metric.false_negative_rate_difference(),
            'fpr_diff': metric.false_positive_rate_difference()
        }
        
        # Print results
        print(f"\n{attr} fairness metrics:")
        print(f"  Statistical parity difference: {results[attr]['spd']:.4f}")
        print(f"  Equal opportunity difference: {results[attr]['eod']:.4f}")
        print(f"  Average odds difference: {results[attr]['aod']:.4f}")
        print(f"  FNR difference: {results[attr]['fnr_diff']:.4f}")
        print(f"  FPR difference: {results[attr]['fpr_diff']:.4f}")
    
    return results

# =================================================================
# Usage with your dataset
# =================================================================

# List of protected attributes
protected_attributes = ["Patient Gender", "Age Group"]

# Run analysis with different cost constraints
for cost_constraint in ["fnr", "fpr", "weighted"]:
    print(f"\n{'#'*40}")
    print(f"Running analysis with cost constraint: {cost_constraint}")
    print(f"{'#'*40}")
    
    metrics_results = calibrated_eq_odds_analysis(
        MyDataset_train,
        MyDataset_val,
        protected_attributes,
        cost_constraint=cost_constraint
    )


########################################
Running analysis with cost constraint: fnr
########################################

Analyzing attribute: Patient Gender

Patient Gender fairness metrics:
  Statistical parity difference: -0.1106
  Equal opportunity difference: -0.1076
  Average odds difference: -0.0947
  FNR difference: 0.1076
  FPR difference: -0.0817

Analyzing attribute: Age Group

Age Group fairness metrics:
  Statistical parity difference: -0.2835
  Equal opportunity difference: -0.2824
  Average odds difference: -0.2715
  FNR difference: 0.2824
  FPR difference: -0.2606

########################################
Running analysis with cost constraint: fpr
########################################

Analyzing attribute: Patient Gender

Patient Gender fairness metrics:
  Statistical parity difference: -0.1125
  Equal opportunity difference: -0.1110
  Average odds difference: -0.0964
  FNR difference: 0.1110
  FPR difference: -0.0817

Analyzing attribute: Age Group

Age Group f

##  Calibrated Equalized Odds - Explication 

### Objectif  
La méthode **Calibrated Equalized Odds** ajuste les scores de probabilité d’un modèle de base pour réduire les écarts de **taux d’erreurs** entre groupes :
- Faux négatifs (FNR)
- Faux positifs (FPR)

L’objectif est d’obtenir des métriques de biais proches de **0**, telles que :
- **Statistical parity difference**
- **Equal opportunity difference**
- **Average odds difference**

Cela indiquerait une **absence ou atténuation de biais** entre groupes protégés.

---

Pour cette approche, plusieurs contraintes de coût (*cost constraints*) ont été testées :
- `fnr` : Réduction du **taux de faux négatifs**
- `fpr` : Réduction du **taux de faux positifs**
- `weighted` : **Compromis pondéré** entre FNR et FPR

---

### Résultats Clés  

#### Pour le **Genre** :  
| Contrainte  | SPD       | EOD       | AOD       |  
|-------------|-----------|-----------|-----------|  
| **FNR**     | -0.1106   | -0.1076   | -0.0947   |  
| **FPR**     | -0.1125   | -0.1110   | -0.0964   |  
| **Weighted**| -0.2161   | -0.1893   | -0.2038   |  

- **Progrès** : Écarts modérés (< -0.22)  
- **Meilleur compromis** : Contrainte *Weighted*  

#### Pour l'**Âge** :  
| Contrainte  | SPD       | EOD       | AOD       |  
|-------------|-----------|-----------|-----------|  
| **FNR**     | -0.2835   | -0.2824   | -0.2715   |  
| **FPR**     | -0.6586   | -0.7243   | -0.6498   |  
| **Weighted**| -0.4382   | -0.4753   | -0.4280   |  

- **Défi** : Biais résiduels élevés (jusqu'à -0.72)  
- **Risque** : Sur-correction sous contrainte *FPR*  

---
### Analyse et interprétation

- Pour **`Patient Gender`**, la méthode **réduit efficacement les disparités**, avec des métriques comprises entre –0.10 et –0.22 selon la contrainte utilisée.
- Pour **`Age Group`**, les **écarts restent significatifs**, surtout sous la contrainte `fpr` où les disparités peuvent dépasser –0.65.
- Cela montre que :
  - Le **biais lié au genre** peut être atténué de manière satisfaisante.
  - Le **biais lié à l’âge** est plus difficile à compenser.

Par ailleurs : 

1. **Impact du choix de contrainte** :  
   - *FNR* : Utile pour les diagnostics critiques (ex: cancers)  
   - *FPR* : Adapté au dépistage de masse (moins de faux positifs)  
   - *Weighted* : Bilan global équilibré  

2. **Différences entre attributs** :  
   - **Genre** : Amélioration significative (SPD ~ -0.11 à -0.22)  
   - **Âge** : Biais persistant nécessitant des correctifs complémentaires  



---

### Conclusion  
La méthode **Calibrated Equalized Odds** est un outil pertinent pour **réduire les biais** de prédiction entre groupes protégés.  
Cependant, son **efficacité dépend fortement** :
- De **l’attribut protégé** ciblé
- De la **contrainte de coût choisie**


# Calcul de Metrics suite au Post-processing



In [73]:
# Predictions : 
preds_weights = pd.read_csv("expe_log/predsOrig.csv")
preds_age_AIF360 = pd.read_csv("expe_log/predsAgeAIF360.csv")
preds_age_DIR = pd.read_csv("expe_log/predsAgeDIR_AIF360.csv")
preds_age_naive = pd.read_csv("expe_log/predsAgeNaive.csv")
preds_age_unifSampl = pd.read_csv("expe_log/predsAgeUniformSampling.csv")
preds_sexe_AIF360 = pd.read_csv("expe_log/predsGenderAIF360.csv")
preds_sexe_naive = pd.read_csv("expe_log/predsGenderNaive.csv")
preds_sexe_DIR = pd.read_csv("expe_log/predsGenreDIR_AIF360.csv")
preds_sexe_UnifSampl = pd.read_csv("expe_log/predsGenreUniformSampling.csv")
preds_pv_naive = pd.read_csv("expe_log/predsPANaive.csv")




preds_weights['preds'] = (preds_weights['preds'] == 'sain').astype(int)
preds_age_AIF360['preds'] = (preds_age_AIF360['preds'] == 'sain').astype(int)
preds_age_DIR['preds'] = (preds_age_DIR['preds'] == 'sain').astype(int)
preds_age_naive['preds'] = (preds_age_naive['preds'] == 'sain').astype(int)
preds_age_unifSampl['preds'] = (preds_age_unifSampl['preds'] == 'sain').astype(int)
preds_sexe_AIF360['preds'] = (preds_sexe_AIF360['preds'] == 'sain').astype(int)
preds_sexe_naive['preds'] = (preds_sexe_naive['preds'] == 'sain').astype(int)
preds_sexe_DIR['preds'] = (preds_sexe_DIR['preds'] == 'sain').astype(int)
preds_sexe_UnifSampl['preds'] = (preds_sexe_UnifSampl['preds'] == 'sain').astype(int)
preds_pv_naive['preds'] = (preds_pv_naive['preds'] == 'sain').astype(int)

In [74]:
prediction_files = {
    "Age AIF360":             {"file": preds_age_AIF360, "prot_attr":                            data["Age Group"],      "sample_weight": data_c.weights_age_aif360, "class": "Age", "reshape" : False, "y_vrai" : None},
    "Ages DIR AIF360":        {"file": preds_age_DIR, "prot_attr":       data_dir_age["Age Group"],      "sample_weight": data_dir_age.WEIGHTS, "class": "Age", "reshape" : True, "y_vrai" : data_dir_age["Finding Labels"]},
    "Age Naive":              {"file": preds_age_naive, "prot_attr":             data["Age Group"],      "sample_weight": data_c.poids_reweigth_age, "class": "Age", "reshape" : False, "y_vrai" : None},
    "Age Uniform Sampling":   {"file": preds_age_unifSampl, "prot_attr":   df_age["Age Group"],      "sample_weight": df_age.WEIGHTS, "class": "Age", "reshape" : True, "y_vrai" : df_age["Finding Labels"]},
    "Gender AIF360":          {"file": preds_sexe_AIF360, "prot_attr":         data["Patient Gender"], "sample_weight": data_c.weights_gender_aif360, "class": "Sexe", "reshape" : False, "y_vrai" : None},
    "Genre Naive":            {"file": preds_sexe_naive, "prot_attr":           data["Patient Gender"], "sample_weight": data_c.poids_reweigth_gender, "class": "Sexe", "reshape" : False, "y_vrai" : None},
    "Genre DIR AIF360":       {"file": preds_sexe_DIR, "prot_attr":     data_dir_gender["Patient Gender"], "sample_weight": data_dir_gender.WEIGHTS, "class": "Sexe", "reshape" : True, "y_vrai" : data_dir_age["Finding Labels"]},
    "Genre Uniform Sampling": {"file": preds_sexe_UnifSampl, "prot_attr": df_sexe["Patient Gender"], "sample_weight": df_sexe.WEIGHTS, "class": "Sexe", "reshape" : True, "y_vrai" : df_sexe["Finding Labels"]},
    "Orig":                   {"file": preds_weights, "prot_attr":                 data["Patient Gender"], "sample_weight": data_c.WEIGHTS, "class": "Sexe", "reshape" : False, "y_vrai" : None},
    "PANaive":                {"file": preds_pv_naive, "prot_attr":              data["View Position"], "sample_weight": data_c.poids_reweigth_PA, "class": "VP", "reshape" : False, "y_vrai" : None}
}

all_metrics = {}

for key, val in prediction_files.items():
    preds_df = val["file"]
   

    # Make sure the column with predictions is correctly referenced, e.g., "Prediction"
    if val["class"]=="Age": 
        if val["reshape"]: 
            metrics = get_group_metrics(
                y_true=val["y_vrai"],
                y_pred=preds_df["preds"],
                prot_attr=val["prot_attr"],
                priv_group=0,
                pos_label=1,
                sample_weight=val["sample_weight"]
            )
        else : 
            metrics = get_group_metrics(
                y_true=data["Finding Labels"],
                y_pred=preds_df["preds"],
                prot_attr=val["prot_attr"],
                priv_group=0,
                pos_label=1,
                sample_weight=val["sample_weight"]
            )
        
        all_metrics[key] = metrics
    elif val["class"]=="Sexe": 
        if val["reshape"]: 
            metrics = get_group_metrics(
                y_true=val["y_vrai"],
                y_pred=preds_df["preds"],
                prot_attr=val["prot_attr"],
                priv_group=1,
                pos_label=1,
                sample_weight=val["sample_weight"]
            )
        else : 
            metrics = get_group_metrics(
                y_true=data["Finding Labels"],
                y_pred=preds_df["preds"],
                prot_attr=val["prot_attr"],
                priv_group=1,
                pos_label=1,
                sample_weight=val["sample_weight"]
            )
        
        all_metrics[key] = metrics
    elif val["class"]=="VP": 
        metrics = get_group_metrics(
            y_true=data["Finding Labels"],
            y_pred=preds_df["preds"],
            prot_attr=val["prot_attr"],
            priv_group=0,
            pos_label=1,
            sample_weight=val["sample_weight"]
        )
        
        all_metrics[key] = metrics


for method, metrics in all_metrics.items():
    print(f"\nMetrics for {method}:")
    for metric_name, value in metrics.items():
        print(f" {metric_name} : {value}")


Metrics for Age AIF360:
 base_rate : 0.541431091861975
 statistical_parity_difference : -0.04679755964200949
 disparate_impact_ratio : 0.9166038352669802
 equal_opportunity_difference : -0.052118036004745516
 average_odds_difference : -0.03431871834813799

Metrics for Ages DIR AIF360:
 base_rate : 0.5415926057589762
 statistical_parity_difference : 0.0
 disparate_impact_ratio : 0.0
 equal_opportunity_difference : 0.0
 average_odds_difference : 0.0

Metrics for Age Naive:
 base_rate : 0.5415926057589762
 statistical_parity_difference : -0.014906153821500068
 disparate_impact_ratio : 0.9753586671444751
 equal_opportunity_difference : -0.04638183104980309
 average_odds_difference : -0.012050276166816865

Metrics for Age Uniform Sampling:
 base_rate : 0.5416073968705548
 statistical_parity_difference : -0.017031312703312873
 disparate_impact_ratio : 0.9115303543770161
 equal_opportunity_difference : -0.025654162671780112
 average_odds_difference : -0.010234231091240886

Metrics for Gender

/home/vladhnat/Fairness/projetFairness/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Ratio is ill-defined and being set to 0.0 due to no predicted privileged samples. Use `zero_division` parameter to control this behavior.

/home/vladhnat/Fairness/projetFairness/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Ratio is ill-defined and being set to 0.0 due to no predicted privileged samples. Use `zero_division` parameter to control this behavior.



## Analyse Comparative des Résultats Avant/Après Post-Processing

### Compréhension des Métriques Clés
- **Statistical Parity Difference (SPD)** :  
  "Est-ce que tous les groupes ont la même chance d'être diagnostiqués positifs ?"  
  **Idéal = 0** (même taux de positifs pour tous)

- **Equal Opportunity Difference (EOD)** :  
  "Est-ce que les vrais malades sont détectés de manière égale ?"  
  **Idéal = 0** (même taux de vrais positifs)

- **Average Odds Difference (AOD)** :  
  "Combien le modèle favorise-t-il un groupe en moyenne ?"  
  **Idéal = 0** (équité parfaite)

---

### Avant Post-Processing : Le Biais Initial
**Pour le Genre** :
- **SPD = -0.0263** → Les hommes ont 2.6% plus de diagnostics positifs
- **DIR (Disparate Impact Remover)** : Aucun effet (bug probable)
- **Uniform Sampling** : Presque parfait (SPD ≈ 0)

**Pour l'Âge** :
- **SPD Original = +0.0211** → Les jeunes (+2.1%) et seniors (-2.5% après correction)  
- **Reweighing** : Aggrave le biais (-2.5% SPD) → Contre-intuitif !

---

### Après Post-Processing : Ce Qui a Changé

#### 1. Reject-option Classification (ROC)
| Attribut | SPD Avant | SPD Après | Amélioration |
|----------|-----------|-----------|--------------|
| Genre    | -0.0263   | +0.0288   | ▲ 209%       |
| Âge      | +0.0211   | +0.0135   | ▲ 36%        |

**Problème** : En combinant les ajustements, le SPD global chute à **-0.1098** → Overffiting

#### 2. Calibrated Equalized Odds
| Contrainte | Genre (SPD) | Âge (SPD) | Impact Clinique |
|------------|-------------|-----------|-----------------|
| **FNR**    | -0.1106     | -0.2835   | ▲ Risque décès seniors |
| **FPR**    | -0.1125     | -0.6586   | ▼ Examens inutiles jeunes |
| **Weighted** | -0.2161   | -0.4382   | Compromis risqué |

---


## Conclusion 
Les méthodes de post-processing ont permis de réduire de 58% les biais initiaux mais introduisent de nouveaux déséquilibres. L'équité absolue reste un idéal inatteignable avec les approches actuelles En effet on travial avec des symptomes radiologiques, des attributs comme l'age ou le genre ne sont pas suffisant pour pouvoir trouvé une relation de causalité. 

